In [8]:
import nltk
import re
import pandas as pd
from nltk.corpus import stopwords
from tqdm.autonotebook import tqdm, trange
from natasha import AddressExtractor
from deeppavlov import configs, build_model

In [9]:
data = pd.read_csv('data/post_data.csv')
data = data.dropna()
data = data.reset_index()
data.drop(['index'], axis=1, inplace=True)

In [10]:
df = data[:]

## Using deepPavlov

In [ ]:
ner_model = build_model(configs.ner.ner_rus_bert)

2019-12-14 12:38:17.421 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /home/caym/.deeppavlov/models/ner_rus_bert/tag.dict]


In [7]:
ner_model('Какой-то шумахер ночью на каршеринге снес мусорку и ушатал соседскую тачку... Перекресток Пинегина и Ткачей. Мне повезло, мою не задел.')

IndexError: index 0 is out of bounds for axis 0 with size 0

In [10]:
f = open('data/pavlov.txt', 'w')
for i in tqdm(list(df['text'])):
    s = ''
    tok = i.split(' ')
    try:
        if len(tok) < 100:
            res = ner_model([i])
            for j in range(len(res[1][0])):
                if res[1][0][j] == 'B-LOC':
                    s += res[0][0][j] + ' '
            f.write(s + '\n')
        else:
            res = ner_model([" ".join(tok[:100])])
            for j in range(len(res[1][0])):
                if res[1][0][j] == 'B-LOC':
                    s += res[0][0][j] + ' '
            f.write(s + '\n')
    except:
        f.write('\n')
f.close()

In [8]:
pavlov = []
f = open('data/pavlov.txt', 'r')
pavlov = [line.strip() for line in f]

In [9]:
data['pavlov'] = pavlov

In [10]:
data.to_csv('data/new_with_street.csv', index=None)

# Пробовали использовать Yargy/Natasha, но хороших результатов не получилось

In [ ]:
morph = pymorphy2.MorphAnalyzer()
stop_words = stopwords.words("russian")

def process_text(text):
    tok = text.split(' ')
    pattern = re.compile("[А-я+0-9-]+")
    words = pattern.findall(" ".join(tok))
    tokens = [word for word in words if word not in stop_words]
    return " ".join(tokens)

In [54]:
extractor = AddressExtractor()
sample = process_text(text)
matches = extractor(text)
for match in matches:
    print(match.fact)

Address(parts=[Settlement(name='Санкт-Петербург', type=None), Street(name='Планерная', type='улица'), Building(number='5', type=None)])


In [37]:
df['text'][8]

'1. Эвакуировали 384 лицей. (в 14:30 всех запустили назад) \n2. Эвакуировали гимназию 405\n3. В 14:00 эвакуировали 98 школу\n4. Эвакуировали гимназию 399\n5. Эвакуировали гимназию 41\n6. 295 гимназию эвакуируют.\n\nЭвакуировали учащихся из школы № 553 Фрунзенского района на улице Ярослава Гашека, 4, корп. 4'

In [38]:
k = 0
f = open('data/street_from_natasha.txt', 'w')
extractor = AddressExtractor()
for i in tqdm(list(df['text'])):
    sample = process_text(i)
    matches = extractor(sample)
    s = ''
    for match in matches:
        for j in range(0, len(match.fact.parts)):
            try:
                if match.fact.parts[j].type is None:
                    try:
                        s += match.fact.parts[j].number + ' '
                        continue
                    except:
                        s += match.fact.parts[j].name + ' '
                        continue
                try:
                    if match.fact.parts[j].number is None:
                        break
                    s += match.fact.parts[j].type + ' ' + match.fact.parts[j].number + ' '
                except:
                    if match.fact.parts[j].name is None:
                        continue
                s += match.fact.parts[j].type + ' ' + match.fact.parts[j].name + ' '
            except:
                continue
    f.write(s + '\n')
    if len(matches)>0:
        k += 1
f.close()